##Get recent tracks per user from Last.fm

In [1]:
import requests
from pprint import pprint
import pylast
import time
import sys

In [2]:
with open("lastfm_apikey.txt", 'r') as laf:
    api_key = laf.readline().strip()
    username = laf.readline().strip()

In [3]:
params = {
           'api_key': api_key,
            'format': 'json',
}

In [4]:
params["user"] = username
params["method"] = "user.getRecentTracks"
params["page"] = '1'
rf = requests.get('http://ws.audioscrobbler.com/2.0/', params=params)

In [5]:
rfj = rf.json()

In [6]:
pprint(rfj["recenttracks"]["track"][10])

{u'album': {u'#text': u'Lioness: Hidden Treasures',
            u'mbid': u'157f27fb-7f7f-4ea9-9674-fd84ed9db612'},
 u'artist': {u'#text': u'Amy Winehouse',
             u'mbid': u'dfe9a7c4-8cf2-47f4-9dcb-d233c2b86ec3'},
 u'date': {u'#text': u'03 Sep 2015, 22:42', u'uts': u'1441320167'},
 u'image': [{u'#text': u'http://img2-ak.lst.fm/i/u/34s/fa0bd630cf0c49bbb753baacccbfc059.png',
             u'size': u'small'},
            {u'#text': u'http://img2-ak.lst.fm/i/u/64s/fa0bd630cf0c49bbb753baacccbfc059.png',
             u'size': u'medium'},
            {u'#text': u'http://img2-ak.lst.fm/i/u/174s/fa0bd630cf0c49bbb753baacccbfc059.png',
             u'size': u'large'},
            {u'#text': u'http://img2-ak.lst.fm/i/u/300x300/fa0bd630cf0c49bbb753baacccbfc059.png',
             u'size': u'extralarge'}],
 u'mbid': u'',
 u'name': u'Like Smoke (feat. Nas)',
 u'streamable': u'0',
 u'url': u'http://www.last.fm/music/Amy+Winehouse/_/Like+Smoke+(feat.+Nas)'}


In [7]:
usernameslist = []
with open("username_urls.txt", 'r') as uuf:
    for line in uuf:
        usernameslist.append(line.strip().split("/")[-1])

In [8]:
print len(usernameslist)#, usernameslist[:5]

13536


In [9]:
usernames = list(set(usernameslist))

In [10]:
print len(usernames)#, usernames[:5]

9335


In [24]:
with open("usernames.txt", 'w') as uf:
    for user in usernames:
        uf.write(user+"\n")

In [4]:
with open("usernames.txt", 'r') as uif:
    usernames = uif.read().split()

In [29]:
mbids = []
time_mbids = []
limit = 200
pages = 20
params["method"] = "user.getRecentTracks"
params["limit"] = str(limit)
count = 0
for user in usernames[9335:]:
    user = user.strip()
    user = user.replace("+", " ")
    print count, user,
    sys.stdout.flush()
    params["user"] = user
    usermbids = []
    usertimembids = []
    time.sleep(0.5)
    for p in range(1, pages+1):
        moveon = False
        while not moveon:
            try:
                params["page"] = str(p)
                urf = requests.get('http://ws.audioscrobbler.com/2.0/', params=params)
                urfj = urf.json()
                recenttracks = urfj["recenttracks"]["track"]
                if len(recenttracks) > 0:
                    for track in recenttracks:
                        mbid = track["mbid"]
                        if mbid != "":
                            usermbids.append(mbid)
                            if "date" in track:
                                usertimembids.append((track["date"]["#text"],track["date"]["uts"], mbid))
                time.sleep(0.1)
                moveon = True
            except requests.exceptions.ConnectionError as e:
                print e,
                time.sleep(3)
    usermbids = list(set(usermbids))
    with open("mbids.txt", 'a') as mf:
        for mb in usermbids:
            mf.write(mb + " ")
        mf.write("\n")
    with open("time_mbids.txt", 'a') as tmf:
        for tm in usertimembids:
            #print tm
            tmf.write(str(tm) + " ")
        tmf.write("\n")
    count += 1

In [27]:
with open("mbids.txt", 'r') as mfr:
    linecount = 0
    for line in mfr:
        linecount += 1
    print linecount

9335


In [81]:
#print mbids[:2]

In [11]:
#print time_mbids[:2]